In [1]:
import numpy as np
from astropy import constants as const
from astropy.coordinates import SkyCoord, Angle, EarthLocation, Longitude, spherical_to_cartesian, cartesian_to_spherical
from astropy.time import Time, TimeDelta
from astropy import units

In [2]:
ra_hrs = 12.1
dec_degs = -42.3
mjd = 55780.1
latitude = Angle('-26d42m11.94986s')
longitude = Angle('116d40m14.93485s')

In [3]:
print(latitude.rad)
print(longitude.rad)

-0.466060844839
2.03628986686


In [4]:
obs_time = Time(mjd, format='mjd', location = (longitude, latitude))
lst_mean = obs_time.sidereal_time('mean')
lst_apparent = obs_time.sidereal_time('apparent')
print(lst_mean.rad)
print(lst_apparent.rad)

1.88387679923
1.88395723805


In [5]:
(lst_mean - lst_apparent).to('deg')

<Angle -0.0046088 deg>

In [6]:
mwa_tools_lst_mean = np.float64(1.8838984)

In [7]:
print(Angle(lst_mean.rad - mwa_tools_lst_mean, unit='rad').to('deg'))
print(Angle(lst_apparent.rad - mwa_tools_lst_mean, unit='rad').to('deg'))

-0d00m04.4555s
0d00m12.1362s


In [8]:
print((lst_mean - Angle(mwa_tools_lst_mean, unit='rad')).to('deg'))
print((lst_apparent - Angle(mwa_tools_lst_mean, unit='rad')).to('deg'))

-0d00m04.4555s
0d00m12.1362s


In [9]:
new_time = obs_time + TimeDelta('-0.8' * units.s)
print(new_time.sidereal_time('apparent').rad)
(new_time.sidereal_time('apparent') - Angle(mwa_tools_lst_mean, unit='rad')).to('deg')
# so to get an apparent LST that matches mwa_tools mean LST, need to add ~ -0.8s

1.88389890112


<Angle 2.87119341e-05 deg>

# -> mean LST disagrees by ~4.5 arcseconds (should we use apparent??)

In [10]:
icrs_coord = SkyCoord(ra=Angle(ra_hrs, unit='hr'), dec=Angle(dec_degs, unit='deg'),
                      obstime=obs_time)
icrs_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (181.5, -42.3)>

In [11]:
icrs_coord.ra.rad, icrs_coord.dec.rad

(3.1677725923697078, -0.7382742735936013)

In [12]:
gcrs_coord = icrs_coord.transform_to('gcrs')
gcrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (181.49525535, -42.3015925)>

In [13]:
gcrs_coord.ra.rad, gcrs_coord.dec.rad

(3.167689782620545, -0.7383020680162412)

In [14]:
# This is the change in ra & dec between icrs & gcrs
print(icrs_coord.ra-gcrs_coord.ra)
print(icrs_coord.dec-gcrs_coord.dec)

0d00m17.0807s
0d00m05.733s


In [15]:
hour_angle_mean = lst_mean.rad - gcrs_coord.ra.rad
hour_angle_apparent = lst_apparent.rad - gcrs_coord.ra.rad

print(hour_angle_mean)
print(hour_angle_apparent)

-1.28381298339
-1.28373254457


In [16]:
# mwa_tools comp2
# It turns out these are not actually used in the return values at all. So don't worry about them
mwa_tools_comp2_ra = 3.1703987
mwa_tools_comp2_dec = -0.73946297

In [17]:
# compare mwa_tools comp2 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp2_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp2_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg09m18.7543s
0deg03m59.4532s


In [18]:
# mwa_tools comp3
mwa_tools_comp3_ra = 3.1676898
mwa_tools_comp3_dec = -0.73830205

In [19]:
# compare mwa_tools comp3 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp3_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp3_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg00m00.0036s
-0deg00m00.0037s


# -> mwa_tools ra_aber/dec_aber (calc3) agrees with GCRS, meaning that it accounts for aberration but not nutation or precession

In [20]:
# The frame radio astronomers call the apparent or current epoch is the
# "true equator & equinox" frame, notated E_gamma in the USNO circular
# astropy doesn't have this frame but it's pretty easy to adapt the CIRS frame
# by modifying the ra to reflect the difference between
# GAST (Grenwich Apparent Sidereal Time) and the earth rotation angle (theta)
def egamma_to_cirs_ra(egamma_ra, time):
    from astropy import _erfa as erfa
    from astropy.coordinates.builtin_frames.utils import get_jd12
    era = erfa.era00(*get_jd12(time, 'ut1'))
    theta_earth = Angle(era, unit='rad')

    assert(isinstance(time, Time))
    assert(isinstance(egamma_ra, Angle))
    gast = time.sidereal_time('apparent', longitude=0)
    cirs_ra = egamma_ra - (gast-theta_earth)
    return cirs_ra

def cirs_to_egamma_ra(cirs_ra, time):
    from astropy import _erfa as erfa
    from astropy.coordinates.builtin_frames.utils import get_jd12
    era = erfa.era00(*get_jd12(time, 'ut1'))
    theta_earth = Angle(era, unit='rad')

    assert(isinstance(time, Time))
    assert(isinstance(cirs_ra, Angle))
    gast = time.sidereal_time('apparent', longitude=0)
    egamma_ra = cirs_ra + (gast-theta_earth)
    return egamma_ra

In [21]:
# now get equivilents of lst, lat in GCRS (rather than precessed) frame:
# effectively current zenith "unprecessed" to J2000 but not undoing aberration
# mwa_tools calls these "lmst2000" and "newarrlat"
# also need gcrs hour angle (ha_gcrs), what they call "ha2000" which is ("lmst2000" - apparent ra)

loc_obj = EarthLocation.from_geodetic(lon=longitude, lat=latitude)

# use CIRS but with a different ra to account for the difference between LST & earth's rotation angle
cirs_ra = egamma_to_cirs_ra(obs_time.sidereal_time('apparent'), obs_time)

assert(cirs_to_egamma_ra(cirs_ra, obs_time) == obs_time.sidereal_time('apparent'))

egamma_zenith_coord = SkyCoord(ra=cirs_ra, dec=latitude, frame='cirs',
                               obstime=obs_time, location = loc_obj)
egamma_zenith_coord

<SkyCoord (CIRS: obstime=55780.1): (ra, dec) in deg
    (107.78961213, -26.70331941)>

In [22]:
# check where it is in altaz (should be near zenith):
egamma_zenith_altaz = egamma_zenith_coord.transform_to('altaz')
egamma_zenith_altaz

<SkyCoord (AltAz: obstime=55780.1, location=(-2559302.5737783727, 5095070.526830904, -2848887.400942108) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (28.60732437, 89.99985797)>

In [23]:
(egamma_zenith_altaz.alt - Angle('90d')).to('deg')

<Angle -0.00014203 deg>

In [24]:
gcrs_zenith = egamma_zenith_altaz.transform_to('gcrs')
gcrs_zenith

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (107.82139279, -26.68249473)>

In [25]:
print(gcrs_zenith.ra.rad)
print(gcrs_zenith.dec.rad)

1.88183830833
-0.465697385751


In [26]:
# This is the change between egamma & gcrs
print(gcrs_zenith.ra - lst_apparent)
print(gcrs_zenith.dec - egamma_zenith_coord.dec)

-0d07m17.0606s
0d01m14.9688s


In [27]:
ha_gcrs = Longitude(gcrs_zenith.ra - gcrs_coord.ra)
ha_gcrs

<Longitude 286.32613744 deg>

In [28]:
ha_gcrs.rad

4.997333832891704

In [29]:
# for these, I used the apparent lst (and aberration corrected ra/decs) from this notebook,
# so that's not a source of error
mwa_tools_comp4_newarrlat = -0.46569739
mwa_tools_comp4_lmst2000 = 1.8818386
mwa_tools_comp4_ha2000 = 4.9973341

In [30]:
# This is what the MWA tools code gives as the change between precessed & not
print(Angle(lst_apparent.rad - mwa_tools_comp4_lmst2000, unit='rad').to('deg'))
print(Angle(latitude.rad - mwa_tools_comp4_newarrlat, unit='rad').to('deg'))

0d07m17.0005s
-0d01m14.9679s


In [31]:
print(Angle(gcrs_zenith.ra.rad - mwa_tools_comp4_lmst2000, unit='rad').to('deg'))
print(Angle(gcrs_zenith.dec.rad - mwa_tools_comp4_newarrlat, unit='rad').to('deg'))
print(Angle(ha_gcrs.rad - mwa_tools_comp4_ha2000, unit='rad').to('deg'))

-0d00m00.0602s
0d00m00.0009s
-0d00m00.0551s


# => Egamma -> CIRS -> CGRS agrees with computation4

In [32]:
# An undocumented feature lets you get the rotation matrix for some Astropy transforms
from astropy import coordinates
from astropy.coordinates.builtin_frames import icrs_fk5_transforms
fk5 = coordinates.FK5()
icrs_fk5_transforms.icrs_to_fk5(None, fk5)

array([[ 1.00000000e+00, -1.11022333e-07, -4.41180450e-08],
       [ 1.11022337e-07,  1.00000000e+00,  9.64779225e-08],
       [ 4.41180343e-08, -9.64779274e-08,  1.00000000e+00]])

In [33]:
# # But not for the ones we want -- this errors:
# from astropy import coordinates
# from astropy.coordinates.builtin_frames import icrs_cirs_transforms
# cirs = coordinates.CIRS(obstime=Time(mjd, format='mjd'))
# icrs_cirs_transforms.icrs_to_cirs(None, cirs)

In [34]:
# now define an antenna position to work on uvw rotation
# the positions used in the mwa_tools code are the same as those used in uvfits antenna positions:
# relative to the array center (expressed in ITRS), but rotated so that the x axis goes through the meridian

from pyuvdata import utils as uvutils
array_center_xyz = np.array([-2559454.08, 5095372.14, -2849057.18])

# in east/north/up frame (relative to array center) in meters: (Nants, 3)
xyz_ants = np.array([[-037.91, 0140.52, 0000.66],[-101.94, 0156.41, 0001.24]])
print(xyz_ants.shape)

lat_lon_alt = uvutils.LatLonAlt_from_XYZ(array_center_xyz)
print(Angle(lat_lon_alt[0]*units.rad).to('deg') - latitude)
print(Angle(lat_lon_alt[1]*units.rad).to('deg') - longitude)
ant_xyz_abs = uvutils.ECEF_from_ENU(xyz_ants.T, lat_lon_alt[0], lat_lon_alt[1], lat_lon_alt[2]).T

ant_xyz_rel_itrs = ant_xyz_abs - array_center_xyz
ant_xyz_rel_rot = uvutils.rotECEF_from_ECEF(ant_xyz_rel_itrs, lat_lon_alt[1])

(2, 3)
0d00m00.0001s
0d00m00.0001s


In [35]:
print(ant_xyz_rel_rot)

[[  63.73518635  -37.91        125.23630472]
 [  71.39382794 -101.94        139.17092739]]


In [36]:
egamma_zenith_xyz = spherical_to_cartesian(1, egamma_zenith_coord.dec, egamma_zenith_coord.ra)
egamma_zenith_xyz

(<Quantity -0.27293726>, <Quantity 0.85062987>, <Quantity -0.44937075>)

In [37]:
egamma_ant_spherical = cartesian_to_spherical(ant_xyz_rel_rot[:, 0], ant_xyz_rel_rot[:, 1], ant_xyz_rel_rot[:, 2])
egamma_ant_spherical

(<Quantity [145.54543655, 186.70133717]>,
 <Latitude [1.03617555, 0.84116482] rad>,
 <Longitude [5.7465945 , 5.32335079] rad>)

In [38]:
cirs_ant_ra = egamma_to_cirs_ra(egamma_ant_spherical[2], obs_time)

cirs_ant_xyz = spherical_to_cartesian(egamma_ant_spherical[0], egamma_ant_spherical[1], cirs_ant_ra)
cirs_ant_xyz

(<Quantity [63.63360223, 71.12102561]>,
 <Quantity [ -38.08026714, -102.13051446]>,
 <Quantity [125.23630472, 139.17092739]>)

In [39]:
cirs_ant_coord = SkyCoord(x=cirs_ant_xyz[0]*units.m, y=cirs_ant_xyz[1]*units.m, z=cirs_ant_xyz[2]*units.m,
                          representation = 'cartesian', frame='cirs',
                          obstime=obs_time)
cirs_ant_coord

<SkyCoord (CIRS: obstime=55780.1): (x, y, z) in m
    [(63.63360223,  -38.08026714, 125.23630472),
     (71.12102561, -102.13051446, 139.17092739)]>

In [40]:
gcrs_ant_coord = cirs_ant_coord.transform_to('gcrs')
gcrs_ant_coord.representation = 'cartesian'
gcrs_ant_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    [(63.77899211,  -38.08136563, 125.16197688),
     (71.28260457, -102.13175108, 139.08732594)]>

In [41]:
mwa_tools_precxyz_xprec = 71.286688
mwa_tools_precxyz_yprec = -102.13304
mwa_tools_precxyz_zprec = 139.08429

In [42]:
# this is how much the input to output changes in mwa_tools (precession + nutation effect)
print(mwa_tools_precxyz_xprec - ant_xyz_rel_rot[1, 0])
print(mwa_tools_precxyz_yprec - ant_xyz_rel_rot[1, 1])
print(mwa_tools_precxyz_zprec - ant_xyz_rel_rot[1, 2])

-0.1071399426444799
-0.1930399991369427
-0.0866373932501574


In [43]:
# This tests agreement between this notebook & mwa_tools
print((gcrs_ant_coord.x[1] - mwa_tools_precxyz_xprec*units.m).to('mm'))
print((gcrs_ant_coord.y[1] - mwa_tools_precxyz_yprec*units.m).to('mm'))
print((gcrs_ant_coord.z[1] - mwa_tools_precxyz_zprec*units.m).to('mm'))

-4.08343381373 mm
1.28892441548 mm
3.03593942368 mm


# The precessed positions match to < 5mm

In [44]:
# try something else. Define antennas in ITRS frame. Then rotate them to GCRS
array_center_coord = SkyCoord(x=array_center_xyz[0]*units.m,
                              y=array_center_xyz[1]*units.m,
                              z=array_center_xyz[2]*units.m,
                              representation = 'cartesian', frame='itrs', obstime=obs_time)
print(array_center_coord)
itrs_coord = SkyCoord(x=ant_xyz_abs[:, 0]*units.m, y=ant_xyz_abs[:, 1]*units.m, z=ant_xyz_abs[:, 2]*units.m,
                      representation = 'cartesian', frame='itrs', obstime=obs_time)
itrs_coord

<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    (-2559454.08, 5095372.14, -2849057.18)>


<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    [(-2559448.81204191, 5095446.11020141, -2848931.94369528),
     (-2559395.03251243, 5095481.69471912, -2848918.00907261)]>

In [45]:
print(itrs_coord.x - array_center_coord.x)
print(itrs_coord.y - array_center_coord.y)
print(itrs_coord.z - array_center_coord.z)

[ 5.26795809 59.04748757] m
[ 73.97020141 109.55471912] m
[125.23630472 139.17092739] m


In [46]:
gcrs_array_center = array_center_coord.transform_to('gcrs')
print(gcrs_array_center)
gcrs_from_itrs_coord = itrs_coord.transform_to('gcrs')
gcrs_from_itrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    (107.82119295, -26.52844311, 6374225.38409199)>


<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    [(107.82081035, -26.52718046, 6374226.42388781),
     (107.82016689, -26.52703835, 6374227.04770571)]>

In [47]:
print(gcrs_array_center)
print(gcrs_array_center.cartesian)
print(gcrs_from_itrs_coord)
print(gcrs_from_itrs_coord.cartesian)

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    (107.82119295, -26.52844311, 6374225.38409199)>
(-1745419.50769061, 5429444.57475286, -2846996.94398546) m
<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    [(107.82081035, -26.52718046, 6374226.42388781),
     (107.82016689, -26.52703835, 6374227.04770571)]>
[(-1745402.73745299, 5429516.84400159, -2846871.72661226),
 (-1745344.09283064, 5429543.69894979, -2846857.85990781)] m


In [48]:
gcrs_rel = (gcrs_from_itrs_coord.cartesian - gcrs_array_center.cartesian).get_xyz().T

print(gcrs_rel)

[[ 16.77023762  72.26924873 125.2173732 ]
 [ 75.41485997  99.12419693 139.08407765]] m


In [49]:
gcrs_lat_lon_alt = uvutils.LatLonAlt_from_XYZ(gcrs_array_center.cartesian.get_xyz().value)
print(gcrs_lat_lon_alt[1])
gcrs_array_center_spherical = gcrs_array_center.spherical
print(gcrs_array_center_spherical.lon - Angle(gcrs_lat_lon_alt[1]*units.rad))
print(gcrs_zenith.ra - Angle(gcrs_lat_lon_alt[1]*units.rad))

zenith_coord = SkyCoord(alt=Angle(90 * units.deg), az=Angle(0 * units.deg),
                        obstime=obs_time, frame='altaz', location=loc_obj)
new_gcrs_zenith = zenith_coord.transform_to('gcrs')
print(new_gcrs_zenith.ra - Angle(gcrs_lat_lon_alt[1]*units.rad))

gcrs_rel_rot = uvutils.rotECEF_from_ECEF(gcrs_rel.value, gcrs_lat_lon_alt[1])

print(gcrs_rel_rot)

1.881834820319562
0d00m00s
0d00m00.7195s
0d00m00.4461s
[[  63.66901083  -38.08335911  125.2173732 ]
 [  71.2873201  -102.13288165  139.08407765]]


In [50]:
print((gcrs_rel_rot[:, 0]*units.m - gcrs_ant_coord.x).to('mm'))
print((gcrs_rel_rot[:, 1]*units.m - gcrs_ant_coord.y).to('mm'))
print((gcrs_rel_rot[:, 2]*units.m - gcrs_ant_coord.z).to('mm'))

[-109.98127684    4.71553421] mm
[-1.99347929 -1.13057712] mm
[55.39632149 -3.248292  ] mm


In [51]:
print(((gcrs_rel_rot[1, 0] - mwa_tools_precxyz_xprec)*units.m).to('mm'))
print(((gcrs_rel_rot[1, 1] - mwa_tools_precxyz_yprec)*units.m).to('mm'))
print(((gcrs_rel_rot[1, 2] - mwa_tools_precxyz_zprec)*units.m).to('mm'))

0.632100400765 mm
0.158347294317 mm
-0.212352578501 mm


# Starting from ITRS does even better: positions match to < 1mm

In [52]:
def mwatools_calcuvw(ha, dec, xyz):
    if xyz.ndim == 1:
        xyz = xyz[np.newaxis, :]
    
    sh = np.sin(ha);
    sd = np.sin(dec);
    ch = np.cos(ha);
    cd = np.cos(dec);

    u = sh * xyz[:, 0] + ch * xyz[:, 1];
    v = -sd * ch * xyz[:, 0] + sd * sh * xyz[:, 1] + cd * xyz[:, 2];
    w =  cd * ch * xyz[:, 0] - cd * sh * xyz[:, 1] + sd * xyz[:, 2];
    return np.array([u, v, w]).T
# matching things up, this code expects relative xyz locations in the 'rotated ECEF' frame expected by uvfits

In [53]:
def calc_uvw_norot(ra, dec, longitude, xyz):
    # same as mwatools_calcuvw but using relative xyz locations in ECEF not rotated frame
    if xyz.ndim == 1:
        xyz = xyz[np.newaxis, :]
    
    ha = Longitude(longitude*units.rad - ra*units.rad).rad
    print(ha)
    
    rot_xyz = uvutils.rotECEF_from_ECEF(xyz, longitude)
    print(rot_xyz)
    
    sr = np.sin(ha);
    sd = np.sin(dec);
    cr = np.cos(ha);
    cd = np.cos(dec);

    u = sr * rot_xyz[:, 0] + cr * rot_xyz[:, 1];
    v = -sd * cr * rot_xyz[:, 0] + sd * sr * rot_xyz[:, 1] + cd * rot_xyz[:, 2];
    w =  cd * cr * rot_xyz[:, 0] - cd * sr * rot_xyz[:, 1] + sd * rot_xyz[:, 2];
    return np.array([u, v, w]).T
 

In [54]:
def phase_uvw(ra, dec, xyz):
    # This code expects relative xyz locations in the same frame that ra/dec are in (e.g. icrs or gcrs)
    
    uvw = np.zeros_like(xyz)
    uvw[:, 0] = (-np.sin(ra) * xyz[:, 0]
                 + np.cos(ra) * xyz[:, 1])
    uvw[:, 1] = (-np.sin(dec) * np.cos(ra) * xyz[:, 0]
                 - np.sin(dec) * np.sin(ra) * xyz[:, 1]
                 + np.cos(dec) * xyz[:, 2])
    uvw[:, 2] = (np.cos(dec) * np.cos(ra) * xyz[:, 0]
                 + np.cos(dec) * np.sin(ra) * xyz[:, 1]
                 + np.sin(dec) * xyz[:, 2])
    return(uvw)

In [55]:
print(ha_gcrs.rad)
print(gcrs_coord.ra.rad)
print(gcrs_coord.dec.rad)

4.99733383289
3.16768978262
-0.738302068016


In [56]:
gcrs_uvw = mwatools_calcuvw(ha_gcrs.rad, gcrs_coord.dec.rad, gcrs_rel_rot)
print(ha_gcrs.rad)
print(gcrs_rel_rot)
gcrs_uvw

4.99733383289
[[  63.66901083  -38.08335911  125.2173732 ]
 [  71.2873201  -102.13288165  139.08407765]]


array([[ -71.80709995,   80.06019817,  -98.06926531],
       [ -97.12282812,   50.38828064, -151.27975942]])

In [57]:
longitude_use = gcrs_zenith.ra.rad
gcrs_uvw_mod1 = calc_uvw_norot(gcrs_coord.ra.rad, gcrs_coord.dec.rad, longitude_use, gcrs_rel.value)
print(gcrs_uvw_mod1)
longitude_use = gcrs_lat_lon_alt[1]
gcrs_uvw_mod2 = calc_uvw_norot(gcrs_coord.ra.rad, gcrs_coord.dec.rad, longitude_use, gcrs_rel.value)
print(gcrs_uvw_mod2)
longitude_use = new_gcrs_zenith.ra.rad
gcrs_uvw_mod3 = calc_uvw_norot(gcrs_coord.ra.rad, gcrs_coord.dec.rad, longitude_use, gcrs_rel.value)
print(gcrs_uvw_mod3)

4.99733383289
[[  63.668878    -38.08358119  125.2173732 ]
 [  71.28696386 -102.1331303   139.08407765]]
[[ -71.8070349    80.0600296   -98.06945056]
 [ -97.12255614   50.38805264 -151.28000997]]
4.99733034488
[[  63.66901083  -38.08335911  125.2173732 ]
 [  71.2873201  -102.13288165  139.08407765]]
[[ -71.8070349    80.0600296   -98.06945056]
 [ -97.12255614   50.38805264 -151.28000997]]
4.99733250752
[[  63.66892847  -38.08349681  125.2173732 ]
 [  71.28709922 -102.13303582  139.08407765]]
[[ -71.8070349    80.0600296   -98.06945056]
 [ -97.12255614   50.38805264 -151.28000997]]


In [58]:
print(((gcrs_uvw_mod1 - gcrs_uvw)*units.m).to('mm'))
print(((gcrs_uvw_mod2 - gcrs_uvw)*units.m).to('mm'))
print(((gcrs_uvw_mod3 - gcrs_uvw)*units.m).to('mm'))
# This means that the difference comes because we're using different longitudes to calculate ha & do the ECEF rotation.
# (gcrs_zenith for the ha & gcrs_lat_lon_alt[1] for the rotation)
# That difference is wrong, we should use the same one.

[[ 0.06505217 -0.16857055 -0.18524627]
 [ 0.27197982 -0.2280002  -0.25055496]] mm
[[ 0.06505217 -0.16857055 -0.18524627]
 [ 0.27197982 -0.2280002  -0.25055496]] mm
[[ 0.06505217 -0.16857055 -0.18524627]
 [ 0.27197982 -0.2280002  -0.25055496]] mm


In [59]:
gcrs_uvw_new = phase_uvw(gcrs_coord.ra.rad, gcrs_coord.dec.rad, gcrs_rel.value)
print(gcrs_coord.ra, gcrs_coord.dec)
gcrs_uvw_new

(<Longitude 181.49525535 deg>, <Latitude -42.3015925 deg>)


array([[ -71.8070349 ,   80.0600296 ,  -98.06945056],
       [ -97.12255614,   50.38805264, -151.28000997]])

In [60]:
print(((gcrs_uvw_new - gcrs_uvw)*units.m).to('mm'))
print(((gcrs_uvw_new - gcrs_uvw_mod1)*units.m).to('mm'))
# So phase_uvw works the same as calc_uvw_norot which would match mwatools_calcuvw if we used the same longitude.
# we'll use phase_uvw in pyuvdata because it's simplest

[[ 0.06505217 -0.16857055 -0.18524627]
 [ 0.27197982 -0.2280002  -0.25055496]] mm
[[-1.42108547e-11  1.42108547e-11  1.42108547e-11]
 [-2.84217094e-11  1.42108547e-11  2.84217094e-11]] mm


In [61]:
mwa_tools_calcuvw_u = -97.122828
mwa_tools_calcuvw_v = 50.388281
mwa_tools_calcuvw_w = -151.27976

In [62]:
print(((gcrs_uvw[1, 0] - mwa_tools_calcuvw_u)*units.m).to('mm'))
print(((gcrs_uvw[1, 1] - mwa_tools_calcuvw_v)*units.m).to('mm'))
print(((gcrs_uvw[1, 2] - mwa_tools_calcuvw_w)*units.m).to('mm'))

-0.000119577293844 mm
-0.000358739107753 mm
0.000584243593948 mm


In [63]:
print(((gcrs_uvw_new[1, 0] - mwa_tools_calcuvw_u)*units.m).to('mm'))
print(((gcrs_uvw_new[1, 1] - mwa_tools_calcuvw_v)*units.m).to('mm'))
print(((gcrs_uvw_new[1, 2] - mwa_tools_calcuvw_w)*units.m).to('mm'))

0.27186024758 mm
-0.228358937129 mm
-0.249970711849 mm


# =>These match exactly, as expected because I copied the code

In [64]:
# Now redo it all but go to icrs instead of gcrs to include abberation
icrs_zenith = egamma_zenith_altaz.transform_to('icrs')
ha_icrs = Longitude(icrs_zenith.ra - icrs_coord.ra)
icrs_array_center = array_center_coord.transform_to('icrs')
icrs_array_center.representation = 'cartesian'
icrs_from_itrs_coord = itrs_coord.transform_to('icrs')
icrs_from_itrs_coord.representation = 'cartesian'

icrs_rel = (np.array([icrs_from_itrs_coord.x.value, icrs_from_itrs_coord.y.value, icrs_from_itrs_coord.z.value]).T
            - np.array([icrs_array_center.x.value, icrs_array_center.y.value, icrs_array_center.z.value]))
icrs_rel_rot = uvutils.rotECEF_from_ECEF(icrs_rel, gcrs_lat_lon_alt[1])
icrs_uvw = mwatools_calcuvw(ha_icrs.rad, icrs_coord.dec.rad, icrs_rel_rot)
icrs_uvw

array([[ -71.81672963,   80.06391822,  -98.06402539],
       [ -97.13904429,   50.39206282, -151.27325458]])

In [65]:
(gcrs_uvw*units.m - icrs_uvw*units.m).to('mm')

<Quantity [[ 9.62968403, -3.72004448, -5.23992214],
           [16.21617341, -3.78217782, -6.50483246]] mm>

# => 10's of mm scale for itrs vs gcrs

In [66]:
# try to figure out what zenith uvws are in current epoch frame -- can they be calculated from ENU?
itrs_lat_lon_alt = uvutils.LatLonAlt_from_XYZ(np.array([array_center_coord.x.value,
                                                        array_center_coord.y.value,
                                                        array_center_coord.z.value]))
itrs_rel = (np.array([itrs_coord.x.value, itrs_coord.y.value, itrs_coord.z.value]).T
            - np.array([array_center_coord.x.value, array_center_coord.y.value, array_center_coord.z.value]))
itrs_rel_rot = uvutils.rotECEF_from_ECEF(itrs_rel, itrs_lat_lon_alt[1])

print(itrs_rel_rot)
print(xyz_ants)

itrs_uvw = mwatools_calcuvw(0, latitude, itrs_rel_rot)
print(itrs_uvw)

# good. So this means that the mwatools calculator,
# which takes coordinates in the weird relative but rotated ECEF frame,
# gives the same answer for zenith phasing as just using ENU positions

[[  63.73518635  -37.91        125.23630472]
 [  71.39382794 -101.94        139.17092739]]
[[ -37.91  140.52    0.66]
 [-101.94  156.41    1.24]]
[[ -37.91        140.52          0.65999992]
 [-101.94        156.41          1.23999991]]


In [67]:
# for phasing the data:
# mwa_tools c code: phase = cexp(I*w*k[chan_ind])
# k[chan_ind] = freq/(VLIGHT/1e6)
# corrected_vis = vis*phase
freq = 150 * units.MHz

bl_uvw = gcrs_uvw[1, :] - gcrs_uvw[0, :]

phase = np.exp(1j * bl_uvw[2]*units.m *freq.to('1/s') / const.c)
phase

<Quantity 0.07978718-0.99681192j>

# Now work out how to "unphase" to drift

In [68]:
def mwatools_calcuvw_unphase(ha, dec, uvw):
    if uvw.ndim == 1:
        uvw = uvw[np.newaxis, :]

    sh = np.sin(ha)
    sd = np.sin(dec)
    ch = np.cos(ha)
    cd = np.cos(dec)

    x = sh * uvw[:, 0] - sd * ch * uvw[:, 1] + cd * ch * uvw[:, 2]
    y = ch * uvw[:, 0] + sd * sh * uvw[:, 1] - cd * sh * uvw[:, 2]
    z = cd * uvw[:, 1] + sd * uvw[:, 2]
    return np.array([x, y, z]).T

In [69]:
# start with gcrs_uvw

# first unphase to get positions in rotECEF frame
uvw_rot_positions = mwatools_calcuvw_unphase(ha_gcrs.rad,
                                             gcrs_coord.dec.rad,
                                             gcrs_uvw)
print(uvw_rot_positions)
print(gcrs_rel_rot)
print(uvw_rot_positions - gcrs_rel_rot)

[[  63.66901083  -38.08335911  125.2173732 ]
 [  71.2873201  -102.13288165  139.08407765]]
[[  63.66901083  -38.08335911  125.2173732 ]
 [  71.2873201  -102.13288165  139.08407765]]
[[ 0.00000000e+00  0.00000000e+00  2.84217094e-14]
 [ 0.00000000e+00 -1.42108547e-14  0.00000000e+00]]


In [70]:
# next rotate them so they can be added to telescope location in frame
uvw_rel_positions = uvutils.ECEF_from_rotECEF(uvw_rot_positions,
                                              gcrs_lat_lon_alt[1])

print(uvw_rel_positions)
print(gcrs_rel)
print(uvw_rel_positions - gcrs_rel.value)

[[ 16.77023762  72.26924873 125.2173732 ]
 [ 75.41485997  99.12419693 139.08407765]]
[[ 16.77023762  72.26924873 125.2173732 ]
 [ 75.41485997  99.12419693 139.08407765]] m
[[-7.10542736e-15  0.00000000e+00  2.84217094e-14]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [71]:
gcrs_uvw_coord = SkyCoord(x=uvw_rel_positions[:, 0] * units.m + gcrs_array_center.cartesian.x,
                          y=uvw_rel_positions[:, 1] * units.m + gcrs_array_center.cartesian.y,
                          z=uvw_rel_positions[:, 2] * units.m + gcrs_array_center.cartesian.z,
                          representation='cartesian',
                          frame='gcrs', obstime=obs_time)

print(gcrs_uvw_coord)
print(gcrs_from_itrs_coord)
print(gcrs_uvw_coord.cartesian - gcrs_from_itrs_coord.cartesian)

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    [(-1745402.73745299, 5429516.84400159, -2846871.72661226),
     (-1745344.09283064, 5429543.69894979, -2846857.85990781)]>
<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    [(107.82081035, -26.52718046, 6374226.42388781),
     (107.82016689, -26.52703835, 6374227.04770571)]>
[(0., 0., 0.), (0., 0., 0.)] m


In [72]:
itrs_uvw_coord = gcrs_uvw_coord.transform_to('itrs')
itrs_uvw_coord.representation = 'cartesian'

print(itrs_uvw_coord)
print(itrs_coord)
print((itrs_uvw_coord.cartesian - itrs_coord.cartesian).get_xyz().to('mm'))

<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    [(-2559448.81205387, 5095446.11017142, -2848931.9436948 ),
     (-2559395.03254594, 5095481.69466586, -2848918.00907071)]>
<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    [(-2559448.81204191, 5095446.11020141, -2848931.94369528),
     (-2559395.03251243, 5095481.69471912, -2848918.00907261)]>
[[-0.01196098 -0.03350759]
 [-0.02998766 -0.05325954]
 [ 0.00048103  0.0019013 ]] mm
